# 💡 How2DDDA: **"Partial Derivatives in Π-Space and Physical Surface Morphology"**

> All code and examples are shared to help researchers, students, and engineers understand the reasoning behind DDDA — and to make it easy to apply dimensional analysis to your own data.  
> This notebook serves as an entry-level guide for teaching, validating physical models, and enabling domain-specific knowledge engineering through data-driven dimensional reasoning.

---

## 🎯 What You'll Learn

**Π-空间偏导与物理曲面形态**

In this notebook, we will walk through the theoretical and computational foundation of **dimensional analysis**, with a focus on the **Buckingham Pi theorem**. You will learn:

1. **The motivation behind dimensional analysis**  
   Understand why we reduce variables and how dimensional consistency enables model generalization.

2. **How to construct the dimensional matrix (D-matrix)**  
   Encode physical units of input quantities using base units and build the D-matrix.

3. **How to compute Π-groups using null space techniques**  
   Discover dimensionless groups by solving linear algebraic equations on the D-matrix.

4. **How to interpret and validate Π-groups**  
   Learn to assess whether derived groups make physical and computational sense.

5. **How to extend the method toward data-driven workflows**  
   Set the stage for further steps in the DDDA pipeline including Pi-group selection, uncertainty quantification, and regime detection.

---

## 👤 Author

- **Name**: Jiashun Pang  
- **Created**: August 2025  
- **Affiliation**: DDDA Project, open research notebook  
- **Notebook Focus**:  
  A hands-on exploration of dimensional analysis — from aggregated raw quantities to symbolic Pi-group discovery and preparation for downstream DDDA tasks.

---

📌 *This notebook is designed to be accessible for learners new to dimensional analysis, while also laying the foundation for advanced applications in the full DDDA pipeline.*


In [5]:
# Section 2 · Synthetic Data I: C_D–Re prototype
# - Defines a reusable generator
# - Creates a sample dataset
# - Saves CSV and draws a quick diagnostic plot

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple, Dict, Any

def simulate_cd_re(
    n_points: int = 500,
    re_min: float = 1e0,
    re_max: float = 1e6,
    re_c: float = 3e5,
    A: float = 24.0,
    p: float = 1.0,
    B: float = 0.2,
    D: float = 0.15,
    sigma_decades: float = 0.35,
    noise_rel: float = 0.03,
    noise_abs: float = 0.0,
    seed: int = 42,
    grid: str = "log",
) -> Tuple[pd.DataFrame, Dict[str, Any]]:
    """
    Generate a synthetic C_D(Re) dataset with a controllable minimum near re_c.

    Clean model:
        C_D_clean(Re) = A / Re^p + B + D * s((log10(Re) - log10(re_c)) / sigma_decades)
        where s(x) = (1 + tanh(x)) / 2  (smooth sigmoid)

    Returns a DataFrame with columns: ["Re", "log10Re", "CD_clean", "CD_noisy"].
    """
    rng = np.random.default_rng(seed)

    if grid == "log":
        Re = np.logspace(np.log10(re_min), np.log10(re_max), n_points)
    elif grid == "lin":
        Re = np.linspace(re_min, re_max, n_points)
    else:
        raise ValueError("grid must be 'log' or 'lin'")

    logRe = np.log10(Re)
    x = (logRe - np.log10(re_c)) / sigma_decades
    sigmoid = 0.5 * (1.0 + np.tanh(x))

    CD_clean = A / (Re ** p) + B + D * sigmoid

    # Noise model: combine relative and absolute components (in quadrature)
    noise_std = np.sqrt((noise_rel * CD_clean) ** 2 + (noise_abs ** 2))
    noise = rng.normal(loc=0.0, scale=noise_std)
    CD_noisy = CD_clean + noise

    df = pd.DataFrame({
        "Re": Re,
        "log10Re": logRe,
        "CD_clean": CD_clean,
        "CD_noisy": CD_noisy
    })

    meta = {
        "re_min": re_min, "re_max": re_max, "re_c": re_c, "A": A, "p": p, "B": B,
        "D": D, "sigma_decades": sigma_decades, "noise_rel": noise_rel,
        "noise_abs": noise_abs, "seed": seed, "grid": grid
    }
    return df, meta

# --- Example usage ---
df, meta = simulate_cd_re(
    n_points=600,
    re_min=1e0,
    re_max=1e6,
    re_c=3e5,          # minimum sits near this Re
    A=24.0,
    p=1.0,
    B=0.2,
    D=0.18,
    sigma_decades=0.35,
    noise_rel=0.025,
    noise_abs=0.0,
    seed=123,
    grid="log"
)

# Save for later sections
df.to_csv("cd_re_synthetic.csv", index=False)
df.head()

,Re,log10Re,CD_clean,CD_noisy
0,1.000000,0.000000,24.200000,23.601582
1,1.023332,0.010017,23.652792,23.435312
2,1.047209,0.020033,23.118060,23.862418
3,1.071643,0.030050,22.595520,22.705094
4,1.096647,0.040067,22.084895,22.592975
